# Programming with Python

## Lecture 06: Type hints, descriptors, metaprogramming

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 22 Mar, 2025

## TypedDict

`TypedDict` declares a dictionary type that expects all of its instances to have a certain set of keys, where each key is associated with a value of a consistent type. This expectation is not checked at runtime but is only enforced by type checkers.

It’s tempting to use TypedDict to protect against errors while handling dynamic data structures like JSON API responses. But correct handling of JSON must be done at runtime, and not with static type checking. For runtime
checking of JSON-like structures using type hints, check out the `pydantic` package.

*References:*

- [TypedDict](https://docs.python.org/3/library/typing.html#typing.TypedDict)
- Fluent Python, Luciano Ramalho

### Practice

Show example 1.

## Type casting

`typing.cast` is used to explicitly tell the type checker that an expression should be treated as a specific type. It does not change the actual value at runtime but is useful for static type checking with tools like `mypy`.

### Practice

Show example 2.

# Generic

**Generic classes** allow defining reusable classes that can handle multiple types while maintaining type safety. They can be defined by using `typing.Generic` abstract base class for generic types.

### Practice

Show example 3.

## Generic static protocols

**Generic static protocols** allow defining interfaces that enforce structure while keeping type flexibility. They are useful when designing generic utilities, factories or contracts.

`typing` module includes several generic static protocols, such as `SupportsAbs`, `SupportsInt`, `SupportsFloat`.

## `typing.runtime_checkable`

The `@runtime_checkable` decorator from `typing` allows protocols to be checked at runtime using `isinstance()` and `issubclass()`. This is useful when working with structural checks similar to abstract base classes.

### Practice

Show example 4.

## Typing map

![Typing map](resources/typing_map.png)

## Descriptors

**Descriptors** are Python objects that implement a method of the descriptor protocol that define the behavior of attribute access. They allow you to customize how attributes are retrieved, set, or deleted. Descriptors are a key part of Python’s object model and are used behind the scenes in properties, methods, static methods, class methods and `super()`. Additionally, their use cases include validation, read-only, computed and lazy attributes.

The default behavior for attribute access is to get, set, or delete the attribute from an object’s dictionary. For instance, `a.x` has a lookup chain starting with `a.__dict__['x']`, then `type(a).__dict__['x']`, and continuing through the method resolution order of `type(a)`. If the looked-up value is an object defining one of the descriptor methods, then Python may override the default behavior and invoke the descriptor method instead. Where this occurs in the precedence chain depends on which descriptor methods were defined.

### Descriptor protocol

The definition of the **descriptor protocol** includes the following methods:

```python
__get__(self, instance, owner=None) -> object
__set__(self, instance, value) -> None
__delete__(self, instance) -> None
__set_name__(self, owner, name)
```

- `self` is the instance of the descriptor.
- `instance` is the instance of the object the descriptor is attached to.
- `owner` is the type of the object the descriptor is attached to.

Descriptors can be classified into:

- **Non-data descriptors:** Only implement `__get__` (e.g. methods).
- **Data descriptors:** Implement `__set__` or `__delete__` along with `__get__` (e.g. properties).

*References*

Throughtout this section the following resources are heavily used:

- [Descriptor Guide](https://docs.python.org/3/howto/descriptor.html)

In [ ]:
class SimpleDescriptor:
    def __init__(self, name=None):
        self.name = name

    def __get__(self, instance, owner=None):
        print(f"__get__ called: self={self}, instance={instance}, owner={owner}")
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        print(f"__set__ called: self={self}, instance={instance}, value={value}")
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        print(f"__delete__ called: self={self}, instance={instance}")
        del instance.__dict__[self.name]

class Point:
    x = SimpleDescriptor("x")
    y = SimpleDescriptor("y")

In [ ]:
obj = Point()

In [ ]:
obj.x = 10 # Triggers __set__

In [ ]:
obj.x # Triggers __get__

In [ ]:
del obj.x # Triggers __delete__

In [ ]:
Point.x # Triggers __get__ with instance set to None

In [ ]:
z = SimpleDescriptor("z")
z

### Read-only data descriptor

A **read-only data descriptor** is a descriptor that allows attribute access but prevents modification. It is commonly used to enforce immutability in class attributes.

Read-only data descritor is defined in the following way:

- Implements `__get__()` to return a value.
- Implements `__set__()` but raises an error to prevent modification.
- Optionally, implements `__delete__()` to prevent deletion.

In [ ]:
class ReadOnly:
    def __init__(self, name=None):
        self.name = name

    def __get__(self, instance, owner=None):
        print(f"__get__ called: self={self}, instance={instance}, owner={owner}")
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None) # Return stored value

    def __set__(self, instance, value):
        raise AttributeError("This attribute is read-only") # Prevent modification

    def __delete__(self, instance):
        raise AttributeError("Cannot delete this attribute") # Prevent deletion

class MyClass:
    attr = ReadOnly("attr")  # Read-only attribute

In [ ]:
obj = MyClass()
obj.attr

In [ ]:
obj.attr = 100

In [ ]:
obj.__dict__

In [ ]:
try:
    obj.attr = 100  # Should raise an error
except AttributeError as e:
    print(e)

In [ ]:
try:
    del obj.attr  # Should raise an error
except AttributeError as e:
    print(e)

### `__dict__` attribute

The `__dict__` attribute in Python is a dictionary that stores an object's writable attributes. It is available for instances and classes that allow dynamic attributes.

In [ ]:
class Rectangle:
    num_of_sides: int = 4
    
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height

In [ ]:
r = Rectangle(1, 2)

print(r.__dict__)
print(type(r))
print(type(r).__dict__) # Same as print(Rectangle.__dict__)

In [ ]:
print(r.width)
print(type(r).num_of_sides)

In [ ]:
# Using __dict__ attribute

print(r.__dict__["width"])
print(type(r).__dict__["num_of_sides"])

### Descriptor invocation

A descriptor can be called directly with `desc.__get__(obj)` or `desc.__get__(None, cls)`.

But it is more common for a descriptor to be invoked automatically from attribute access.

Descriptors are invoked by the [`__getattribute__(self, name)`](https://docs.python.org/3/reference/datamodel.html#object.__getattribute__) method, which returns the attribute value or raises an `AttributeError` exception if an attribute is not found.

The expression `obj.x` looks up the attribute `x` in the chain of namespaces for `obj`. If the search finds a descriptor outside of the instance `__dict__`, its `__get__()` method is invoked according to the precedence rules listed below.

1. **`__getattribute__` method:** First the object's `__getattribute__` method is called, which is responsible for attribute access.
2. **Data Descriptors:** If the attribute is found in the class (or its parent classes) and is a data descriptor, the descriptor's `__get__` method is called.
3. **Instance Dictionary:** If the attribute is found in the object's `__dict__`, that value is returned.
4. **Non-Data Descriptors**: If the attribute is found in the class (or its parent classes) and is a non-data descriptor (implements only `__get__`), the descriptor's `__get__` method is called.
5. **Class Dictionary:** If the attribute is found in the class's `__dict__` (or its parent classes), that value is returned.
6. **`__getattr__` method:** If the attribute is not found anywhere else and the object has a `__getattr__` method, that method is called.
7. **`AttributeError`:** If all the above steps fail, Python raises an `AttributeError`.

In [ ]:
class DataDescriptor:
    """A descriptor that implements both __get__ and __set__"""
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner=None):
        print(f"2. DataDescriptor.__get__ called for {self.name}")
        return f"DataDescriptor value for {self.name}"
        
    def __set__(self, instance, value):
        print(f"DataDescriptor.__set__ called for {self.name} with value {value}")

class NonDataDescriptor:
    """A descriptor that implements only __get__"""
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner=None):
        print(f"4. NonDataDescriptor.__get__ called for {self.name}")
        return f"NonDataDescriptor value for {self.name}"
    
class OverridingNoGetDescriptor:
    """A descriptor that implements only __set__"""
    def __init__(self, name):
        self.name = name
        
    def __set__(self, instance, value):
        print(f"OverridingNoGetDescriptor.__set__ called for {self.name} with value {value}")

class MyClass:
    data_desc_attr = DataDescriptor("data_desc") # Data descriptor
    non_data_desc_attr = NonDataDescriptor("non_data_desc") # Non-data descriptor
    over_no_get_attr = OverridingNoGetDescriptor("overriding_no_get_attr") # Overriding descriptor with no __get__
    class_attr = "class attribute" # Regular class attribute
    
    def __init__(self):
        self.instance_attr = "instance attribute" # Regular instance attribute
        
    def __getattr__(self, name):
        print(f"6. __getattr__ called for {name}")
        return f"__getattr__ value for {name}"
        
    def __getattribute__(self, name):
        print(f"1. __getattribute__ called for {name}")
        result = super().__getattribute__(name)
        print(f"__getattribute__ {result=}")
        return result

#### Lookup chain for different attributes

In [ ]:
obj = MyClass()

print("Accessing data descriptor:")
print(obj.data_desc_attr)

print("\nAccessing instance attribute:")
print(obj.instance_attr)

print("\nAccessing non-data descriptor:")
print(obj.non_data_desc_attr)

print("\nAccessing class attribute:")
print(obj.class_attr)

print("\nAccessing non-existent attribute:")
print(obj.doesnt_exist_attr)

print("\nAccessing overriding with no __get__ descriptor:")
print(obj.over_no_get_attr)